In [2]:
import time as tm
import re
import pyvista as pv
import os
import numpy as np
import pandas as pd
#from tqdm import tqdm

import os
#import PVGeo
import rasterio

import gemgis


In [3]:
def read_GOCAD_ts_test(tsfile):
    import re
 
    fid = open(tsfile, 'r')
    vrtx = []
    trgl = []
    #split atom line by by space, make vertex at that point in VRTX list
    #go back and get atomlist[1]  VRTXLIST[atomlist[1]-1]
    
    for line in fid:
        if 'VRTX' in line:
            l_input = re.split('[\s*]', line)
            temp = np.array(l_input[2:5])
            #temp = np.array(l_input[2:6])
            vrtx.append(temp.astype(np.float))

        if 'ATOM' in line:
            l_input = re.split('[\s*]', line)
            vertex_id = l_input[1]
            vertex_id_atom = l_input[2]
            #print(len(vrtx), vertex_id, vertex_id_atom)
            vrtx.append(vrtx[ int(vertex_id_atom) -1])

        if 'TRGL' in line:
            l_input = re.split('[\s*]', line)
            temp = np.array(l_input[1:4])
            trgl.append(temp.astype(np.int))
            
    vrtx = np.asarray(vrtx)
    trgl = np.asarray(trgl)
 
    return vrtx, trgl

def check_GOCAD_ts(tsfile):
    import re
 
    fid = open(tsfile, 'r')
    tslist = tsfile.split('\\')
    crs_dict = {"NAME":None,"AXIS_NAME":None,"AXIS_UNIT":None,"ZPOSITIVE":None}
    check_dist = {"SURFACE":tslist[-1],"NAME":None,"COORD":0,"TFACE":0,"PVRTX":0,"VRTX":0,"TRGL":0,"ATOM":0,"CRS":crs_dict, "COLOR":None}

    for index, line in enumerate(fid):
        ##HEADER
        if 'color:' in line:
            strcolor = line.split(':')
            tricolor = re.split('[\s*]', strcolor[1])
            tricolor.pop()
            tricolor[0] = float(tricolor[0]) #rgb 0-1 colors R
            tricolor[1] = float(tricolor[1]) #rgb 0-1 colors G
            tricolor[2] = float(tricolor[2]) #rgb 0-1 colors B
            check_dist["COLOR"] = tricolor
        if 'name:' in line:
            strname = line.split(':')
            usename = strname[1].replace("\n",'')
            check_dist["NAME"] = usename
        if 'NAME ' in line and "AXIS" not in line:
            strname = re.split('[\s*]', line)
            crs = strname[1].replace("\n",'')
        if 'AXIS_NAME' in line:
            axis_name = re.split('[\s*]', line)
            axis_name.pop()
        if 'AXIS_UNIT' in line:
            axis_unit = re.split('[\s*]', line)
            axis_unit.pop()
        if 'ZPOSITIVE' in line:
            zpositive = re.split('[\s*]', line)[1].replace("\n",'')
        if 'END_ORIGINAL_COORDINATE' in line:
            crs_dict["NAME"] = crs
            crs_dict["AXIS_NAME"] = axis_name
            crs_dict["AXIS_UNIT"] = axis_unit
            crs_dict["ZPOSITIVE"] = zpositive

        
        #####
        ####Data
        if 'TFACE' in line:
            check_dist["TFACE"] += 1
        if 'COORDINATE_SYSTEM' in line:
            check_dist["COORD"] += 1
        if 'ATOM' in line:
            check_dist["ATOM"] += 1
        if 'PVRTX' in line:
            check_dist["PVRTX"] += 1
        if 'VRTX' in line:
            check_dist["VRTX"] += 1
        if 'TRGL' in line:
            check_dist["TRGL"] += 1
            
    return check_dist

In [4]:
neoprot_ordovician_list = []
late_mesoprot_list = []
middle_mesoprot_list = []
archean_earlymesoprot_list = []

faults_list = []

for root, dirs, files in os.walk(r'H:\GDP00022\GOCAD\Neoprot-Ordovician'):
    for file in files:
        print(file)
        if '.ts' in file:
            #print(file)
            #dlist = file.split('_')
            neoprot_ordovician_list.append(file)


0025_O-i_InnaminckaFormation_2.ts
0050_O2_PostDelamerianBasalt.ts
0075_EO_UndiffCambro-OrdovMetaseds.ts
0100_EO4_KalladeinaFormation.ts
0125_EOu_DullingariGroup_2.ts
0150_EOd24_PreSynDelamerianFelsicInts.ts
0175_EOd25_postDelamerianFelsicInts.ts
0200_EOd26_postDelamerianMaficInts.ts
0225_EOd27_BritishEmpireGranite.ts
0245_Ea_KangarooIslandGroup.ts
0275_Ek_KanmantooGroup_bathy.ts
0300_E-u_KulyongVolcanics.ts
0300_E-u_TableHillVolcanics.ts
0325_E-m_MooracoochieVolcanics_2.ts
0350_Ek3_MaficVolcsNormanvilleGroup.ts
0375_E2_MoorilyannaFm.ts
0400_E-j_JenaBasalt_2.ts
0425_Eo8_TruroVolcanics.ts
0450_Eo9_FelsicIntermedVolcanics.ts
0475_Ek4_VolcanicsNormanvilleGroup.ts
0499_Em4_ReliefSandstone.ts
0525_Em5_ObservatoryHillFm.ts
0550_Eo10_WinultaFm.ts
0575_Eo11_RamsayLimestone.ts
0601_Eo12_UratannaFm.ts
0625_Eo13_BillyCreekFm.ts
0650_NE1_UndiffNeoprot-CambMetaseds.ts
0675_NO1_UndiffNeoprot-OrdovMetaseds.ts
0700_LN_UndiffProterozoic_2.ts
0725_N_UndiffNeoproterozoic_5.ts
0750_Nox_BoucautVolcanics.ts


In [ ]:
surface_list = []
v_list = []
metadata_list = []
t_list = []
face_list = []

mesh_list = []


p = pv.Plotter(notebook=True)
              
if 1 == 1:

    for index, d in enumerate(neoprot_ordovician_list ):
        #print("Neoprot:",d)
            
        try:
            v, t = read_GOCAD_ts_test(r'H:\GDP00022\GOCAD\Neoprot-Ordovician' + "\\" + str(d) )
            metadata = check_GOCAD_ts(r'H:\GDP00022\GOCAD\Neoprot-Ordovician' + "\\" + str(d) )
        except Except as gocadE:
            print(gocadE)

        t = t -1

        tri_index = t

        faces = np.empty_like(tri_index, shape=(tri_index.shape[0], 4) )
        faces[:, 0] = 3
        faces[:, 1:] = tri_index

        surf2 = pv.PolyData(v, faces)

        p.add_mesh(surf2.copy(), name=str(d), color=metadata["COLOR"][0:3])

        mesh_list.append(surf2)
        v_list.append(v)
        metadata_list.append(metadata)
        surface_list.append(d)
        face_list.append(faces)

    dataset_list =[]    
    
    for i, s in enumerate(surface_list):
        dataset = pd.DataFrame({'Group':'Neoprot-Ordovician','Surface': s, 'X': v_list[i][:, 0], 'Y': v_list[i][:, 1], 'Z': v_list[i][:, 2], 'SG': -1   })
        dataset["Name"] = metadata_list[i]["NAME"]
        dataset["CRS"] = str(metadata_list[i]["CRS"])
        dataset["Color"] = str(metadata_list[i]["COLOR"])

        dataset_list.append(dataset)

    dfNeoprot_Ordovician = pd.concat(dataset_list)    

p.show_grid(color='black')
p.set_background(color='white')
p.show(cpos=(1,-4,5))


<ipython-input-3-2d5b4e4b975e>:15: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  vrtx.append(temp.astype(np.float))
<ipython-input-3-2d5b4e4b975e>:27: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  trgl.append(temp.astype(n

In [ ]:
p.show(cpos=(1,-4,5))